## Observations and Insights 

In [52]:
# Dependencies and Setup
import matplotlib.pyplot as plt
%matplotlib widget
import pandas as pd
import scipy.stats as st
import numpy as np

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single dataset
data_df = pd.merge(mouse_metadata,study_results, how= 'outer', on=['Mouse ID'])

# Display the data table for preview
data_df.head()

ModuleNotFoundError: No module named 'matplotlib.pyplot.plot'; 'matplotlib.pyplot' is not a package

In [35]:
# Checking the number of mice.
total_mice = data_df["Mouse ID"].nunique()
print(f" The total number of mice in the study is: {total_mice}")

 The total number of mice in the study is: 249


In [36]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
duplicates = data_df[data_df.duplicated(["Mouse ID", "Timepoint"])]
duplicates

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
909,g989,Propriva,Female,21,26,0,45.000000,0
911,g989,Propriva,Female,21,26,5,47.570392,0
913,g989,Propriva,Female,21,26,10,49.880528,0
915,g989,Propriva,Female,21,26,15,53.442020,0
917,g989,Propriva,Female,21,26,20,54.657650,1


In [37]:
# Optional: Get all the data for the duplicate mouse ID. 


In [38]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
new_df = data_df.loc[data_df["Mouse ID"]!= "g989"]
new_df.head()

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1


In [39]:
# Checking the number of mice in the clean DataFrame.
total_clean = new_df["Mouse ID"].nunique()
print(f"the number of mice are: {total_clean}")

the number of mice are: 248


## Summary Statistics

In [40]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
mean_regimen = new_df.groupby('Drug Regimen').mean()["Tumor Volume (mm3)"]
median_regimen = new_df.groupby('Drug Regimen').median()["Tumor Volume (mm3)"]
variance_regimen = new_df.groupby('Drug Regimen').var()["Tumor Volume (mm3)"]
sd_regimen = new_df.groupby('Drug Regimen').std()["Tumor Volume (mm3)"]
sem_regimen = new_df.groupby('Drug Regimen').median()["Tumor Volume (mm3)"]
sem_regimen
# This method is the most straighforward, creating multiple series and putting them all together at the end.



Drug Regimen
Capomulin    41.557809
Ceftamin     51.776157
Infubinol    51.820584
Ketapril     53.698743
Naftisol     52.509285
Placebo      52.288934
Propriva     50.446266
Ramicane     40.673236
Stelasyn     52.431737
Zoniferol    51.818479
Name: Tumor Volume (mm3), dtype: float64

In [41]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
stats_table = pd.DataFrame({"Mean": mean_regimen, "Median":median_regimen, "Variance":variance_regimen, "Standard Deviation": sd_regimen, "SEM": sem_regimen})
stats_table
# This method produces everything in a single groupby function


,Mean,Median,Variance,Standard Deviation,SEM
Drug Regimen,,,,,
Capomulin,40.675741,41.557809,24.947764,4.994774,41.557809
Ceftamin,52.591172,51.776157,39.290177,6.268188,51.776157
Infubinol,52.884795,51.820584,43.128684,6.567243,51.820584
Ketapril,55.235638,53.698743,68.553577,8.279709,53.698743
Naftisol,54.331565,52.509285,66.173479,8.134708,52.509285
Placebo,54.033581,52.288934,61.168083,7.821003,52.288934
Propriva,52.320930,50.446266,43.852013,6.622085,50.446266
Ramicane,40.216745,40.673236,23.486704,4.846308,40.673236
Stelasyn,54.233149,52.431737,59.450562,7.710419,52.431737


## Bar and Pie Charts

In [42]:
# Generate a bar plot showing the total number of mice for each treatment throughout the course of the study using pandas. 
treatments = data_df.groupby("Drug Regimen")
mice = treatments["Mouse ID"].count()

chart = mice.plot(kind='bar', title='total number of mice')
plt.xlabel(treatments)
plt.ylabel('mice')
plt.show();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [43]:
# Generate a bar plot showing the total number of mice for each treatment throughout the course of the study using pyplot.
x = new_df["Drug Regimen"].value_counts()
plt.bar(x.index, x.values, color='green', alpha=0.8, align="center")
plt.xticks(rotation=50);
plt.ylabel('mice')
plt.xlabel('treatments')

Text(0.5, 0, 'treatments')

In [44]:
# Generate a pie plot showing the distribution of female versus male mice using pandas
#count the mice by sex
sex = new_df['Sex'].value_counts()
#plot the pie chart using the index of the sex "male & female"
sex.plot(kind='pie', labels=sex.index, subplots=True, autopct="%1.1f%%")
plt.show()

In [45]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot
colors = ["yellow","green"]
plt.pie(sex.values, labels=sex.index, colors=colors,
        autopct="%1.1f%%", shadow=True, startangle=90);

## Quartiles, Outliers and Boxplots

In [46]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse
mouse_id = new_df.groupby("Mouse ID")
max_timepoint = mouse_id.Timepoint.max()
max_timepoint = max_timepoint.reset_index()

# Merge this group df with the original dataframe to get the tumor volume at the last timepoint
merge_df = max_timepoint.merge(new_df, how='right', on=['Timepoint', 'Mouse ID'])
merge_df

,Mouse ID,Timepoint,Drug Regimen,Sex,Age_months,Weight (g),Tumor Volume (mm3),Metastatic Sites
0,a203,45,Infubinol,Female,20,23,67.973419,2
1,a251,45,Infubinol,Female,21,25,65.525743,1
2,a262,45,Placebo,Female,17,29,70.717621,4
3,a275,45,Ceftamin,Female,20,28,62.999356,3
4,a366,30,Stelasyn,Female,16,29,63.440686,1
...,...,...,...,...,...,...,...,...
1875,z969,20,Naftisol,Male,9,30,57.898778,2
1876,z969,25,Naftisol,Male,9,30,63.145652,2
1877,z969,30,Naftisol,Male,9,30,65.841013,3
1878,z969,35,Naftisol,Male,9,30,69.176246,4


In [47]:
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    
    
    # add subset 
    
    
    # Determine outliers using upper and lower bounds
    

In [48]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest


## Line and Scatter Plots

In [55]:
# Generate a line plot of time point versus tumor volume for a mouse treated with Capomulin
line_df = merge_df.loc[merge_df["Mouse ID"] == "b128",:]
#line_df.head()
x_axis = line_df["Timepoint"]
tumorsize = line_df["Tumor Volume (mm3)"]

plt.plot(x_axis, tumorsize, color='green', marker="o")
plt.show()

In [61]:
# Generate a scatter plot of mouse weight versus average tumor volume for the Capomulin regimen
scatter_df =merge_df.groupby(['Mouse ID']).mean()
scatter_df.head()

#plt.scatter(scatter_df['Weight (g)'],scatter_df['Tumor Volume (mm3)'],s=100, color="blue")
#plt.xlabel('Timepoint')
#plt.ylabel('Tumor Volume (mm3)')

#plt.show()

,Timepoint,Age_months,Weight (g),Tumor Volume (mm3),Metastatic Sites
Mouse ID,,,,,
a203,22.5,20.0,23.0,56.312708,1.100000
a251,22.5,21.0,25.0,55.237095,0.800000
a262,22.5,17.0,29.0,58.611631,2.300000
a275,22.5,20.0,28.0,52.770965,1.200000
a366,15.0,16.0,29.0,54.639906,0.142857


## Correlation and Regression

In [ ]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
